<a href="https://colab.research.google.com/github/MateoIbaniez/ML---Analisis-de-datos/blob/main/lightgbm_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aclaraciones:

No se puede correr todo el notebook de una. Se debe correr todo hasta despues de haber agragado la segunda nueva feature (que esta en el apartado nuevas features), alli se guarda lo ya obtenido y se reinicia el entorno. Se continua (cargando lo guardado) y se termina con las nuevas features. Por las dudas conviene aca volver a guardar, reiniciar y cargar. Contiunuar con el shodan, el split entre train y validation el encodeo y por ultimo normalizar. Una vez aqui ya se esta listo para buscar hiper parametros y predecir



#Imports


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install -U scikit-learn

In [ ]:
from sklearn.preprocessing import TargetEncoder

In [ ]:
import numpy as np
import scipy as sc

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
pip install category_encoders

In [ ]:
import pandas as pd
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import sklearn as sk
import re
import category_encoders as ce
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier


#Carga de los datos y drop

In [ ]:
data = pd.read_parquet('/content/drive/MyDrive/data tp ml/train.parq')

Dado a que el "watcher_as_name" y "watcher_as_num" (lo mismo con attacker), representan ambos al dato AS (Autonomous System), pero uno como string y el otro como numero, procedo a dropear, las columnas "watcher_as_name" y "attacker_as_name". Explicacion de que es AS: Un sistema autónomo (AS) es una red o grupo de redes muy grande con una única política de enrutamiento. A cada AS se le asigna un único ASN, que es un número que identifica al AS. (explicacionn obtenida del sitio de cloudflare).

In [ ]:
data.drop(axis=1, columns=['attacker_as_num', "watcher_as_num"], inplace=True)

Vemos si hay filas duplicadas. En caso de que todos los elementos de la fila sean repetidos

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data

,attack_time,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label
0,2023-07-31 07:17:51+00:00,DE,Host Europe GmbH,TR,Murat Aktas,http:exploit,0,6466,0
1,2023-07-31 07:17:51+00:00,DE,Host Europe GmbH,TR,Murat Aktas,http:spam,0,6466,0
2,2023-07-31 07:17:49+00:00,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:bruteforce,2,4637,0
3,2023-07-31 07:17:49+00:00,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:spam,2,4637,0
4,2023-07-31 07:17:49+00:00,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:exploit,2,4637,0
...,...,...,...,...,...,...,...,...,...
61629671,2023-08-24 21:14:59+00:00,US,INMOTION,US,ATT-INTERNET4,http:bruteforce,9606,191439,0
61629673,2023-08-24 21:14:59+00:00,US,INMOTION,US,ATT-INTERNET4,http:scan,9606,191439,0
61629674,2023-08-24 21:14:59+00:00,US,INMOTION,US,ATT-INTERNET4,http:exploit,9606,191439,0
61629681,2023-08-24 21:14:46+00:00,US,NAMECHEAP-NET,US,WOW,http:scan,40383,193446,0


In [ ]:
#data.dropna(how = 'any', inplace=True)
columnas = data.columns
dicc = {}

for col in columnas:
  dicc[col] = data[col].isnull().values.any()

dicc

{'attack_time': False,
 'watcher_country': True,
 'watcher_as_name': True,
 'attacker_country': True,
 'attacker_as_name': True,
 'attack_type': False,
 'watcher_uuid_enum': False,
 'attacker_ip_enum': False,
 'label': False}

In [ ]:
data.watcher_country = data.watcher_country.astype('object')
data.attacker_country = data.attacker_country.astype('object')

data.watcher_country.fillna ( 'nan', inplace = True)
data.attacker_country.fillna('nan', inplace = True)

data.watcher_country = data.watcher_country.astype('category')
data.attacker_country = data.attacker_country.astype('category')

data.watcher_as_name = data.watcher_as_name.astype('object')
data.attacker_as_name = data.attacker_as_name.astype('object')

data.watcher_as_name.fillna ( 'nan', inplace = True)
data.attacker_as_name.fillna('nan', inplace = True)

data.watcher_as_name = data.watcher_as_name.astype('category')
data.attacker_as_name = data.attacker_as_name.astype('category')

Veo cuantas cosas distintas hay en cada categoria. No me fijo cuantos attackt_time hay xq es obvio que muchos.

In [ ]:
#categorias = {}

#categorias['watcher_country'] = len(list(data.watcher_country.unique()))
# categorias['attacker_country'] = len(list(data.attacker_country.unique()))
# categorias['attack_type'] = len(list(data.attack_type.unique()))
# categorias['attacker_as_num'] = len(list(data.attacker_as_num.unique()))
# categorias['watcher_as_num'] = len(list(data.watcher_as_num.unique()))

# categorias

#Acomodar la data


Con lo analizado pasamos los tipos de ataque de 15 a 6

*   http:spam
*   http:scan
*   http:exploit
*   ssh:bruteforce
*   http:bruteforce
*   otro

In [ ]:
tipos_freq = data.attack_type.value_counts().reset_index()
tipos_freq.columns = ['tipo', 'cantidad']
tipos_freq

,tipo,cantidad
0,http:spam,12971937
1,http:scan,9378467
2,http:exploit,7191852
3,ssh:bruteforce,4533447
4,http:bruteforce,4220069
5,windows:bruteforce,568301
6,pop3/imap:bruteforce,440027
7,tcp:scan,334790
8,unknown:unknown,323418
9,sip:bruteforce,224943


In [ ]:
primeros_5 = list(tipos_freq.tipo.unique()[:5])
cinco_attack_types = data['attack_type']

def agrupar_attack_types(x):
  tipo = str(x)
  if tipo in primeros_5:
    return tipo
  else:
    return 'otro_tipo'

cinco_attack_types = cinco_attack_types.map(agrupar_attack_types)

In [ ]:
data.attack_type =  cinco_attack_types

In [ ]:
data.attack_type = data.attack_type.astype('category')

In [ ]:
data.attack_type.unique()

['http:exploit', 'http:spam', 'http:bruteforce', 'http:scan', 'otro_tipo', 'ssh:bruteforce']
Categories (6, object): ['http:bruteforce', 'http:exploit', 'http:scan', 'http:spam', 'otro_tipo',
                         'ssh:bruteforce']

En watche_country nos quedamos los 5 paises que mas frecuencia

In [ ]:
tipos_freq = data.watcher_country.value_counts().reset_index()
tipos_freq.columns = ['pais', 'cantidad']
tipos_freq

,pais,cantidad
0,US,13935028
1,DE,6247878
2,FR,3556434
3,GB,2199930
4,NL,2117352
...,...,...
126,MV,10
127,BB,9
128,IM,9
129,MQ,2


In [ ]:
primeros_5 = list(tipos_freq.pais.unique()[:5])
cinco_watcher_country = data['watcher_country']

def agrupar_watcher_countrys(x):
  pais = str(x)
  if pais in primeros_5:
    return pais
  else:
    return 'otro_pais'

cinco_watcher_country = cinco_watcher_country.map(agrupar_watcher_countrys)

In [ ]:
data.watcher_country =  cinco_watcher_country

In [ ]:
data.watcher_country = data.watcher_country.astype('category')

En attacker_counrty me quedo los 16 primeros paises

In [ ]:
tipos_freq = data.attacker_country.value_counts().reset_index()
tipos_freq.columns = ['pais', 'cantidad']
tipos_freq

,pais,cantidad
0,US,11037930
1,DE,3374327
2,SG,2468276
3,CN,2451386
4,NL,2132369
...,...,...
210,ER,16
211,NE,13
212,FO,10
213,GD,6


In [ ]:
primeros_15 = list(tipos_freq.pais.unique()[:14])
quince_attacker_country = data['attacker_country']
def agrupar_attacker_countrys(x):
  pais = str(x)
  if pais in primeros_15:
    return pais
  else:
    return 'otro_pais'

quince_attacker_country = quince_attacker_country.map(agrupar_attacker_countrys)

In [ ]:
data.attacker_country =  quince_attacker_country

In [ ]:
data.attacker_country = data.attacker_country.astype('category')

Ajustamos las timezones

In [ ]:
pasar_dias = {
  '730' : '1',
  '731' : '2'
}
formato = 798
for i in range(3, 32):
  pasar_dias[str(formato+i)] = str(i)



In [ ]:
time = data.attack_time
time1 = time[:20000000]
time2 = time[20000000:]

In [ ]:
def recortar_y_formatear_time(x):
  fecha = str(x)
  #mes_dia = fecha[6:10]
  horas_min_seg = fecha[11:14]
  #mes_dia = re.sub("[-]","", mes_dia)
  #dia = pasar_dias[mes_dia]
  horas_min_seg = re.sub("[:]","", horas_min_seg)

  return int(horas_min_seg)


In [ ]:
time11 = time1.map(recortar_y_formatear_time)

In [ ]:
time22 = time2.map(recortar_y_formatear_time)

In [ ]:
timefinal = pd.concat([time11, time22])
data.attack_time = timefinal

In [ ]:
data.attack_time = data.attack_time.astype('int32')

In [ ]:
data

,attack_time,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label
0,7,DE,Host Europe GmbH,otro_pais,Murat Aktas,http:exploit,0,6466,0
1,7,DE,Host Europe GmbH,otro_pais,Murat Aktas,http:spam,0,6466,0
2,7,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:bruteforce,2,4637,0
3,7,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:spam,2,4637,0
4,7,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:exploit,2,4637,0
...,...,...,...,...,...,...,...,...,...
61629671,21,US,INMOTION,US,ATT-INTERNET4,http:bruteforce,9606,191439,0
61629673,21,US,INMOTION,US,ATT-INTERNET4,http:scan,9606,191439,0
61629674,21,US,INMOTION,US,ATT-INTERNET4,http:exploit,9606,191439,0
61629681,21,US,NAMECHEAP-NET,US,WOW,http:scan,40383,193446,0


In [ ]:
a = list(tipos_freq.pais.unique()[:14])
a

['US',
 'DE',
 'SG',
 'CN',
 'NL',
 'FR',
 'IN',
 'GB',
 'RU',
 'CA',
 'VN',
 'BR',
 'ID',
 'HK']

In [ ]:
def poner_ceros(x):
  y = str(x)
  if y == 'nan':
    return 'unknown'
  else:
    return y

data.watcher_as_name = data.watcher_as_name.map(poner_ceros)
data.attacker_as_name = data.attacker_as_name.map(poner_ceros)

data.watcher_as_name = data.watcher_as_name.astype('object')
data.watcher_as_name = data.watcher_as_name.astype('category')

data.attacker_as_name = data.attacker_as_name.astype('object')
data.attacker_as_name = data.attacker_as_name.astype('category')

#nuevas features


**primera feature**

In [ ]:
paises = data[['attacker_country', 'watcher_country']]
paises['cantidad'] = 1
aux = paises.groupby( ['attacker_country', 'watcher_country'] )['cantidad'].sum()
paises = aux.reset_index()
paises

<ipython-input-37-607cd6eb2ed8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paises['cantidad'] = 1


,attacker_country,watcher_country,cantidad
0,BR,DE,76473
1,BR,FR,51931
2,BR,GB,35621
3,BR,NL,31033
4,BR,US,212086
...,...,...,...
85,otro_pais,FR,747289
86,otro_pais,GB,494997
87,otro_pais,NL,445762
88,otro_pais,US,2955087


In [ ]:
data = data.merge(paises, how = 'left')
cols = data.columns
cols

Index(['attack_time', 'watcher_country', 'watcher_as_name', 'attacker_country',
       'attacker_as_name', 'attack_type', 'watcher_uuid_enum',
       'attacker_ip_enum', 'label', 'cantidad'],
      dtype='object')

In [ ]:
data.columns = ['attack_time', 'watcher_country', 'watcher_as_name', 'attacker_country',
       'attacker_as_name', 'attack_type', 'watcher_uuid_enum',
       'attacker_ip_enum', 'label', 'cantidad_ataques_ya_recibidos_x_ese_pais']
data

,attack_time,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label,cantidad_ataques_ya_recibidos_x_ese_pais
0,7,DE,Host Europe GmbH,otro_pais,Murat Aktas,http:exploit,0,6466,0,1394749
1,7,DE,Host Europe GmbH,otro_pais,Murat Aktas,http:spam,0,6466,0,1394749
2,7,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:bruteforce,2,4637,0,1015799
3,7,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:spam,2,4637,0,1015799
4,7,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:exploit,2,4637,0,1015799
...,...,...,...,...,...,...,...,...,...,...
40290303,21,US,INMOTION,US,ATT-INTERNET4,http:bruteforce,9606,191439,0,4401326
40290304,21,US,INMOTION,US,ATT-INTERNET4,http:scan,9606,191439,0,4401326
40290305,21,US,INMOTION,US,ATT-INTERNET4,http:exploit,9606,191439,0,4401326
40290306,21,US,NAMECHEAP-NET,US,WOW,http:scan,40383,193446,0,4401326


In [ ]:
data.cantidad_ataques_ya_recibidos_x_ese_pais = data.cantidad_ataques_ya_recibidos_x_ese_pais.astype('int32')

**segunda feature**

Cantidad de ataques provenientes de los distintos paises, segun la hora del dia

EJ: a las 4 de la maniana US tiene 10000. O sea se registraron 10000 ataques de US en esa hora.

In [ ]:
paises_x_hora = data[['attack_time', 'attacker_country']]
paises_x_hora['cantidad_ataques_pais_x_hora'] = 1
paises_x_hora = paises_x_hora.groupby(['attack_time', 'attacker_country'])['cantidad_ataques_pais_x_hora'].sum()
paises_x_hora

<ipython-input-41-bdbab9adaab2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paises_x_hora['cantidad_ataques_pais_x_hora'] = 1


attack_time  attacker_country
0            BR                   25465
             CA                   38933
             CN                  104476
             DE                  123396
             FR                   81673
                                  ...  
23           RU                   49840
             SG                   86814
             US                  383532
             VN                   31908
             otro_pais           341516
Name: cantidad_ataques_pais_x_hora, Length: 360, dtype: int64

In [ ]:
paises_x_hora = paises_x_hora.to_frame().reset_index()
paises_x_hora

,attack_time,attacker_country,cantidad_ataques_pais_x_hora
0,0,BR,25465
1,0,CA,38933
2,0,CN,104476
3,0,DE,123396
4,0,FR,81673
...,...,...,...
355,23,RU,49840
356,23,SG,86814
357,23,US,383532
358,23,VN,31908


In [ ]:
data = data.merge(paises_x_hora, how = 'left')

In [ ]:
data.cantidad_ataques_pais_x_hora = data.cantidad_ataques_pais_x_hora.astype('int32')

In [ ]:
data.columns = ['attack_time', 'watcher_country', 'watcher_as_name', 'attacker_country',
       'attacker_as_name', 'attack_type', 'watcher_uuid_enum',
       'attacker_ip_enum', 'label', 'cantidad_ataques_ya_recibidos_x_ese_pais',
       'cantidad_ataques_pais_x_hora']

In [ ]:
import pickle
filename = 'data_half_90_10_nulos_como_unknown.pickle'
pickle.dump(data, open(filename, "wb"))

In [ ]:
import pickle
filename = '/content/drive/MyDrive/data_half_90_10_nulos_como_unknown.pickle'
data = pickle.load(open(filename, "rb"))

In [ ]:
data.info()

**tercera feature**

Ahora otra feature, cuantas veces el AS del watcher, recibio ataques del AS del attacker

In [ ]:
AS = data[['attacker_as_name', 'watcher_as_name']]
AS['cantidad_ataques_AS_to_AS'] = 1
aux = AS.groupby(['attacker_as_name', 'watcher_as_name'])['cantidad_ataques_AS_to_AS'].sum()
AS = aux.reset_index()
AS

<ipython-input-7-8ef5911c2a0f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AS['cantidad_ataques_AS_to_AS'] = 1


,attacker_as_name,watcher_as_name,cantidad_ataques_AS_to_AS
0,''It Fruit'' S.r.l.,-Reserved AS,0
1,''It Fruit'' S.r.l.,1&1 Versatel Deutschland GmbH,0
2,''It Fruit'' S.r.l.,10 Science Park Road,0
3,''It Fruit'' S.r.l.,"10dencehispahard, S.L.",0
4,''It Fruit'' S.r.l.,123NET,0
...,...,...,...
22234499,zicom Next Spolka Z Ograniczona Odpowiedzialno...,webhoster.de AG,0
22234500,zicom Next Spolka Z Ograniczona Odpowiedzialno...,wilhelm.tel GmbH,0
22234501,zicom Next Spolka Z Ograniczona Odpowiedzialno...,xTom,0
22234502,zicom Next Spolka Z Ograniczona Odpowiedzialno...,xTom GmbH,0


In [ ]:
data = data.merge(AS, how = 'left')
data

,attack_time,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label,cantidad_ataques_ya_recibidos_x_ese_pais,cantidad_ataques_pais_x_hora,cantidad_ataques_AS_to_AS
0,7,DE,Host Europe GmbH,otro_pais,Murat Aktas,http:exploit,0,6466,0,1394749,400399,2262
1,7,DE,Host Europe GmbH,otro_pais,Murat Aktas,http:spam,0,6466,0,1394749,400399,2262
2,7,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:bruteforce,2,4637,0,1015799,152801,218
3,7,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:spam,2,4637,0,1015799,152801,218
4,7,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:exploit,2,4637,0,1015799,152801,218
...,...,...,...,...,...,...,...,...,...,...,...,...
40290303,21,US,INMOTION,US,ATT-INTERNET4,http:bruteforce,9606,191439,0,4401326,419116,864
40290304,21,US,INMOTION,US,ATT-INTERNET4,http:scan,9606,191439,0,4401326,419116,864
40290305,21,US,INMOTION,US,ATT-INTERNET4,http:exploit,9606,191439,0,4401326,419116,864
40290306,21,US,NAMECHEAP-NET,US,WOW,http:scan,40383,193446,0,4401326,419116,9311


In [ ]:
data.cantidad_ataques_AS_to_AS.fillna ( 0, inplace = True)
data.cantidad_ataques_AS_to_AS = data.cantidad_ataques_AS_to_AS.astype('int32')

Hacemos otras 2 features nuevas

In [ ]:
country_AS_attack_type = data[['attacker_country', 'attacker_as_name', 'attack_type']]
aux = country_AS_attack_type.groupby(['attacker_country', 'attacker_as_name'])['attack_type'].value_counts()
df_aux = aux.to_frame()


In [ ]:
df_aux.columns = ['freq']
df_aux = df_aux.reset_index()

In [ ]:
df_aux

**cuarta feature**

Esta es una feature, por cada ataque con cierto pais y AS de attacker se muestra, ademas del tipo del ataque, las veces que se hizo ese ataque con ese pais y AS. la columna "freq"

In [ ]:
df_frecuencia_country_AS_attackType = df_aux.loc[df_aux.freq > 0]

In [ ]:
data = data.merge(df_frecuencia_country_AS_attackType, how = 'left')
data

,attack_time,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label,cantidad_ataques_ya_recibidos_x_ese_pais,cantidad_ataques_pais_x_hora,cantidad_ataques_AS_to_AS,freq
0,7,DE,Host Europe GmbH,otro_pais,Murat Aktas,http:exploit,0,6466,0,1394749,400399,2262,42298
1,7,DE,Host Europe GmbH,otro_pais,Murat Aktas,http:spam,0,6466,0,1394749,400399,2262,44845
2,7,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:bruteforce,2,4637,0,1015799,152801,218,68967
3,7,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:spam,2,4637,0,1015799,152801,218,180990
4,7,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:exploit,2,4637,0,1015799,152801,218,133188
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40290303,21,US,INMOTION,US,ATT-INTERNET4,http:bruteforce,9606,191439,0,4401326,419116,864,4349
40290304,21,US,INMOTION,US,ATT-INTERNET4,http:scan,9606,191439,0,4401326,419116,864,18688
40290305,21,US,INMOTION,US,ATT-INTERNET4,http:exploit,9606,191439,0,4401326,419116,864,7964
40290306,21,US,NAMECHEAP-NET,US,WOW,http:scan,40383,193446,0,4401326,419116,9311,136675


**quinta feature**

Contamos la cantidad de veces que ya se repitio la combinacion pais-as del attacker

In [ ]:
country_AS_attack_type = data[['attacker_country', 'attacker_as_name']]
country_AS_attack_type['cantidad_pais_as_attacker'] = 1
aux = country_AS_attack_type.groupby(['attacker_country', 'attacker_as_name'])['cantidad_pais_as_attacker'].sum()
df_aux = aux.to_frame()
df_aux

<ipython-input-14-3276e9ce44ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_AS_attack_type['cantidad_pais_as_attacker'] = 1


cantidad_pais_as_attacker
attacker_country attacker_as_name                                                             
BR               ''It Fruit'' S.r.l.                                                         0
                 ''it Fruit'' S.r.l.                                                         0
                 (aq) networks limited                                                       0
                 - CONECTMAIS                                                                8
                 -Reserved AS                                                                0
...                                                                                        ...
otro_pais        wirulink                                                                    6
                 xTom GmbH                                                                  63
                 xneelo                                                                   3186
                 zain-as                                                                   452
                 zicom Next Spolka Z Ograniczona Odpowiedzialnoscia                          6

[120360 rows x 1 columns]

In [ ]:
#Alivio el dataset sacando los que tienen rep = 0
df_aux = df_aux.loc[df_aux.cantidad_pais_as_attacker > 0]
df_aux

cantidad_pais_as_attacker
attacker_country attacker_as_name                                                             
BR               - CONECTMAIS                                                                8
                 1TELECOM SERVICOS DE TECNOLOGIA EM INTERNET LTDA                          549
                 3D COIMBRA TELECOMUNICACOES EIRELI                                        221
                 3WLINK INTERNET LTDA                                                      566
                 4K Telecom Internet Ltda - ME                                              12
...                                                                                        ...
otro_pais        wirulink                                                                    6
                 xTom GmbH                                                                  63
                 xneelo                                                                   3186
                 zain-as                                                                   452
                 zicom Next Spolka Z Ograniczona Odpowiedzialnoscia                          6

[8829 rows x 1 columns]

In [ ]:
df_aux = df_aux.reset_index()
df_aux

,attacker_country,attacker_as_name,cantidad_pais_as_attacker
0,BR,- CONECTMAIS,8
1,BR,1TELECOM SERVICOS DE TECNOLOGIA EM INTERNET LTDA,549
2,BR,3D COIMBRA TELECOMUNICACOES EIRELI,221
3,BR,3WLINK INTERNET LTDA,566
4,BR,4K Telecom Internet Ltda - ME,12
...,...,...,...
8824,otro_pais,wirulink,6
8825,otro_pais,xTom GmbH,63
8826,otro_pais,xneelo,3186
8827,otro_pais,zain-as,452


In [ ]:
data = data.merge(df_aux, how = 'left')

In [ ]:
data.freq.fillna( 0, inplace = True)
data.cantidad_ataques_pais_x_hora.fillna( 0, inplace = True)
data.cantidad_pais_as_attacker.fillna( 0, inplace = True)


data.attacker_country = data.attacker_country.astype('category')
data.freq = data.freq.astype('int32')
data.cantidad_pais_as_attacker = data.cantidad_pais_as_attacker.astype('int32')

In [ ]:
import pickle
filename = 'data_90_10_nulos_unknown.pickle'
pickle.dump(data, open(filename, "wb"))

#cargarmos el df ya hecho y el shodan

In [ ]:
import pickle
filename = '/content/drive/MyDrive/data_90_10_nulos_unknown.pickle'
data = pickle.load(open(filename, "rb"))

In [ ]:
shodan = pd.read_csv('/content/drive/MyDrive/data tp ml/shodan_df_hashed.csv')

In [ ]:
def poner_unknowns(x):
  y = str(x)
  if y == '{}':
    return 'unknown'
  else:
    return y

shodan.shodan_info = shodan.shodan_info.map(poner_unknowns)

In [ ]:
shodan.shodan_info = shodan.shodan_info.astype('category')
shodan.attacker_ip_enum = shodan.attacker_ip_enum.astype('int32')

In [ ]:
data = data.merge(shodan)

In [ ]:
data.cantidad_ataques_AS_to_AS.fillna ( 0, inplace = True)
data.freq.fillna( 0, inplace = True)
data.cantidad_pais_as_attacker.fillna ( 0, inplace = True)
data.cantidad_ataques_pais_x_hora.fillna( 0, inplace = True)
data.cantidad_pais_as_attacker.fillna ( 0, inplace = True)


#Separar la data en Train y Validation

In [ ]:
freq_ip1 = data.loc[data['label'] == 1]['attacker_ip_enum']
freq_ip1 = freq_ip1.value_counts().reset_index()
freq_ip1.columns = ['ip', 'frecuencia']
freq_ip1

,ip,frecuencia
0,75769,63717
1,90822,43821
2,19555,43355
3,193313,33983
4,157556,30938
...,...,...
2885,13055,2
2886,78692,2
2887,43034,2
2888,113269,2


In [ ]:
freq_ip0 = data.loc[data['label'] == 0]['attacker_ip_enum']
freq_ip0 = freq_ip0.value_counts().reset_index()
freq_ip0.columns = ['ip', 'frecuencia']
freq_ip0

,ip,frecuencia
0,63518,227203
1,165283,226254
2,117926,220344
3,57799,194696
4,115212,186185
...,...,...
145364,111973,1
145365,187054,1
145366,103566,1
145367,183206,1


In [ ]:
from random import seed
import random

In [ ]:
total = freq_ip1.frecuencia.sum() + freq_ip0.frecuencia.sum()
total

40290308

In [ ]:
freq_ip1.frecuencia = freq_ip1.frecuencia / (total * 0.2)
ips1 = list(freq_ip1.ip.unique())

seed(7)

porcentaje1 = 0
total_train1 = 0.02
ips_validation1 = []

while(porcentaje1 < total_train1):
   ip1 = random.choice(ips1)
   ips1.remove(ip1)
   porcentaje1 = porcentaje1 + freq_ip1.loc[freq_ip1.ip == ip1].frecuencia.values[0]
   ips_validation1.append(ip1)


In [ ]:
porcentaje1

0.02203048435370612

In [ ]:
freq_ip0.frecuencia = freq_ip0.frecuencia / (total * 0.2)
ips0 = list(freq_ip0.ip.unique())

seed(7)

porcentaje0 = 0
total_train0 = 0.98

ips_validation0 = []

while(porcentaje0 < total_train0):
  ip = random.choice(ips0)
  ips0.remove(ip)
  porcentaje0 = porcentaje0 + freq_ip0.loc[freq_ip0.ip == ip].frecuencia.values[0]
  ips_validation0.append(ip)

In [ ]:
porcentaje0

0.9804648304003996

In [ ]:
ips_validation1.extend(ips_validation0)
X_validation = data.loc[data.attacker_ip_enum.isin(ips_validation1)]
y_validation  = X_validation['label']
X_validation.drop(['attacker_ip_enum', 'label'], axis=1, inplace=True)

<ipython-input-22-b15663008846>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_validation.drop(['attacker_ip_enum', 'label'], axis=1, inplace=True)


In [ ]:
ips1.extend(ips0)
df_train = data.loc[data.attacker_ip_enum.isin(ips1)]
y_train  = df_train['label']


#Encodeo

OHE para 'type_attack'

In [ ]:
df_train.drop(['label', 'attacker_ip_enum'], axis=1, inplace=True)

<ipython-input-24-015a9ea9d0a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(['label', 'attacker_ip_enum'], axis=1, inplace=True)


In [ ]:
class OHE:
  def __init__(self):
    self.categorias = []

  def fit(self, feature):
    self.categorias.extend( list(feature.unique()) )

  def fit_transform(self, feature):
    self.fit(feature)
    nuevas_columnas = []
    for valor in self.categorias:
      nuevas_columnas.append( (feature == valor).astype('int8') )
    return nuevas_columnas

  def transform(self, feature):
    nuevas_columnas = []
    for valor in self.categorias:
      nuevas_columnas.append( (feature == valor).astype('int8') )
    return nuevas_columnas


In [ ]:
ohe_encoder = OHE()

In [ ]:
cols = ohe_encoder.fit_transform(df_train.attack_type)

In [ ]:
len(cols)

6

In [ ]:
tipos_de_ataques = list(df_train.attack_type.unique())

i = 0
for ataque in tipos_de_ataques:
  df_train[ataque] = cols[i]
  i = i + 1


<ipython-input-29-3ff934a60752>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[ataque] = cols[i]
<ipython-input-29-3ff934a60752>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[ataque] = cols[i]
<ipython-input-29-3ff934a60752>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

In [ ]:
df_train.drop('attack_type', axis=1, inplace=True)

<ipython-input-30-981f3406020a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop('attack_type', axis=1, inplace=True)


In [ ]:
cols_validation = ohe_encoder.transform(X_validation.attack_type)

In [ ]:
len(cols_validation)

6

In [ ]:
i = 0
for ataque in tipos_de_ataques:
  X_validation[ataque] = cols_validation[i]
  i = i + 1


<ipython-input-33-0cd096b2ec06>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_validation[ataque] = cols_validation[i]
<ipython-input-33-0cd096b2ec06>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_validation[ataque] = cols_validation[i]
<ipython-input-33-0cd096b2ec06>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [ ]:
X_validation.drop('attack_type', axis=1, inplace=True)

<ipython-input-34-2dc5fed06c6e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_validation.drop('attack_type', axis=1, inplace=True)


Encodeamos a 'watcher_country' con un OHE:

In [ ]:
ohe_watcher_country = OHE()

In [ ]:
cols = ohe_watcher_country.fit_transform(df_train.watcher_country)

In [ ]:
paises = list(df_train.watcher_country.unique())

i = 0
for pais in paises:
  df_train['watcher_' + pais] = cols[i]
  i = i + 1


<ipython-input-37-36a437fa2699>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['watcher_' + pais] = cols[i]
<ipython-input-37-36a437fa2699>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['watcher_' + pais] = cols[i]
<ipython-input-37-36a437fa2699>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [ ]:
df_train.drop('watcher_country', axis=1, inplace=True)

<ipython-input-38-2c3c02a112c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop('watcher_country', axis=1, inplace=True)


In [ ]:
cols_valid = ohe_watcher_country.transform(X_validation.watcher_country)

In [ ]:

i = 0
for pais in paises:
  X_validation['watcher_' + pais] = cols_valid[i]
  i = i + 1


<ipython-input-40-c4e18b537f92>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_validation['watcher_' + pais] = cols_valid[i]
<ipython-input-40-c4e18b537f92>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_validation['watcher_' + pais] = cols_valid[i]
<ipython-input-40-c4e18b537f92>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [ ]:
X_validation.drop('watcher_country', axis=1, inplace=True)

<ipython-input-41-e39cada1920e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_validation.drop('watcher_country', axis=1, inplace=True)


Hacemos un binary encoding para 'attacker_country'

In [ ]:
encoder_attacker_country = ce.BinaryEncoder()

In [ ]:
encoder_attacker_country.fit(df_train['attacker_country'])

BinaryEncoder(cols=['attacker_country'],
              mapping=[{'col': 'attacker_country',
                        'mapping':      attacker_country_0  attacker_country_1  attacker_country_2  \
 1                    0                   0                   0   
 2                    0                   0                   1   
 3                    0                   0                   1   
 4                    0                   1                   0   
 5                    0                   1                   0   
 6                    0                   1                   1   
 7                    0                   1                   1   
 8                    1                   0                   0   
 9                    1                   0                   0   
 10                   1                   0                   1   
 11                   1                   0                   1   
 12                   1                   1                   0   
 13                   1                   1                   0   
 14                   1                   1                   1   
 15                   1                   1                   1   
-1                    0                   0                   0   
-2                    0                   0                   0   

     attacker_country_3  
 1                    1  
 2                    0  
 3                    1  
 4                    0  
 5                    1  
 6                    0  
 7                    1  
 8                    0  
 9                    1  
 10                   0  
 11                   1  
 12                   0  
 13                   1  
 14                   0  
 15                   1  
-1                    0  
-2                    0  }])

In [ ]:
aux = encoder_attacker_country.transform(df_train['attacker_country'])

In [ ]:
df_train =pd.concat([df_train, aux], axis=1)

In [ ]:
df_train.attacker_country_0 = df_train.attacker_country_0.astype('int8')
df_train.attacker_country_1 = df_train.attacker_country_1.astype('int8')
df_train.attacker_country_2 = df_train.attacker_country_2.astype('int8')
df_train.attacker_country_3 = df_train.attacker_country_3.astype('int8')

In [ ]:
df_train.drop('attacker_country', axis=1, inplace=True)

In [ ]:
aux = encoder_attacker_country.transform(X_validation['attacker_country'])

In [ ]:
X_validation =pd.concat([X_validation, aux], axis=1)

In [ ]:
X_validation.drop('attacker_country', axis=1, inplace=True)

In [ ]:
X_validation.attacker_country_0 = X_validation.attacker_country_0.astype('int8')
X_validation.attacker_country_1 = X_validation.attacker_country_1.astype('int8')
X_validation.attacker_country_2 = X_validation.attacker_country_2.astype('int8')
X_validation.attacker_country_3 = X_validation.attacker_country_3.astype('int8')

attacker country

In [ ]:
# encoder_attacker_country = TargetEncoder(target_type = 'binary' ,cv = 10, random_state=2)

# attacker_country_encoded = encoder_attacker_country.fit_transform(df_train['attacker_country'].to_numpy().reshape(-1, 1), y_train)
# df_train.attacker_country = attacker_country_encoded
# df_train.attacker_country = df_train.attacker_country.astype('float32')

# X_attacker_country_encoded = encoder_attacker_country.transform(X_validation['attacker_country'].to_numpy().reshape(-1, 1))
# X_validation.attacker_country = X_attacker_country_encoded
# X_validation.attacker_country = X_validation.attacker_country.astype('float32')

# #df_train.drop('attacker_country', axis=1, inplace=True)
# #X_validation.drop('attacker_country', axis=1, inplace=True)

Attacker as name

In [ ]:
encoder_attacker_as_name  = TargetEncoder(target_type = 'binary' ,cv =10, random_state=1)
attacker_as_name_encoded  = encoder_attacker_as_name.fit_transform(df_train['attacker_as_name'].to_numpy().reshape(-1, 1) , y_train)
df_train.attacker_as_name = attacker_as_name_encoded
df_train.attacker_as_name = df_train.attacker_as_name.astype('float32')

In [ ]:
X_attacker_as_name_encoded = encoder_attacker_as_name.transform(X_validation['attacker_as_name'].to_numpy().reshape(-1, 1))
X_validation.attacker_as_name = X_attacker_as_name_encoded
X_validation.attacker_as_name = X_validation.attacker_as_name.astype('float32')

watcher as name

In [ ]:
encoder_watcher_as_name  = TargetEncoder(target_type = 'binary' ,cv = 10, random_state=1)
watcher_as_name_encoded  = encoder_watcher_as_name.fit_transform(df_train['watcher_as_name'].to_numpy().reshape(-1, 1), y_train)
df_train.watcher_as_name = watcher_as_name_encoded
df_train.watcher_as_name = df_train.watcher_as_name.astype('float32')

In [ ]:
X_watcher_as_name_encoded = encoder_watcher_as_name.transform(X_validation['watcher_as_name'].to_numpy().reshape(-1, 1))
X_validation.watcher_as_name = X_watcher_as_name_encoded
X_validation.watcher_as_name = X_validation.watcher_as_name.astype('float32')

shodan

In [ ]:
shodan_encoder = TargetEncoder(target_type = 'binary' ,cv = 10, random_state=1)
shodan_info_encoded = shodan_encoder.fit_transform(df_train['shodan_info'].to_numpy().reshape(-1, 1) , y_train)
df_train.shodan_info = shodan_info_encoded
df_train.shodan_info = df_train.shodan_info.astype('float32')

In [ ]:
X_shodan_info_encoded    = shodan_encoder.transform(X_validation['shodan_info'].to_numpy().reshape(-1, 1))
X_validation.shodan_info = X_shodan_info_encoded
X_validation.shodan_info = X_validation.shodan_info.astype('float32')

Attack_time

In [ ]:
time_encoder = TargetEncoder(target_type = 'binary' ,cv = 10, random_state=1)
time_encoded = time_encoder.fit_transform(df_train['attack_time'].to_numpy().reshape(-1, 1) , y_train)
df_train.attack_time = time_encoded
df_train.attack_time = df_train.attack_time.astype('float32')

In [ ]:
X_time_encoded    = time_encoder.transform(X_validation['attack_time'].to_numpy().reshape(-1, 1))
X_validation.attack_time = X_time_encoded
X_validation.attack_time = X_validation.attack_time.astype('float32')

watcher_uuid_enum

In [ ]:
uuid_encoder = TargetEncoder(target_type = 'binary' ,cv = 10, random_state=1)
uuid_encoded = uuid_encoder.fit_transform(df_train['watcher_uuid_enum'].to_numpy().reshape(-1, 1) , y_train)
df_train.watcher_uuid_enum = uuid_encoded
df_train.watcher_uuid_enum = df_train.watcher_uuid_enum.astype('float32')

In [ ]:
X_uuid_encoded   = uuid_encoder.transform(X_validation['watcher_uuid_enum'].to_numpy().reshape(-1, 1))
X_validation.watcher_uuid_enum = X_uuid_encoded
X_validation.watcher_uuid_enum = X_validation.watcher_uuid_enum.astype('float32')

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32212139 entries, 0 to 40290307
Data columns (total 26 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   attack_time                               float32
 1   watcher_as_name                           float32
 2   attacker_as_name                          float32
 3   watcher_uuid_enum                         float32
 4   cantidad_ataques_ya_recibidos_x_ese_pais  int32  
 5   cantidad_ataques_pais_x_hora              int32  
 6   cantidad_ataques_AS_to_AS                 int32  
 7   freq                                      int32  
 8   cantidad_pais_as_attacker                 int32  
 9   shodan_info                               float32
 10  http:exploit                              int8   
 11  http:spam                                 int8   
 12  http:bruteforce                           int8   
 13  http:scan                                 int8   
 14  

#encodear nuevas features

as_to_as

In [ ]:
as_to_as_encoder = TargetEncoder(target_type = 'binary' ,cv = 10, random_state=1)
as_to_as_encoded = as_to_as_encoder.fit_transform(df_train['cantidad_ataques_AS_to_AS'].to_numpy().reshape(-1, 1) , y_train)
df_train.cantidad_ataques_AS_to_AS  = as_to_as_encoded
df_train.cantidad_ataques_AS_to_AS  = df_train.cantidad_ataques_AS_to_AS .astype('float32')

X_as_to_as_encoded = as_to_as_encoder.transform(X_validation['cantidad_ataques_AS_to_AS'].to_numpy().reshape(-1, 1))
X_validation.cantidad_ataques_AS_to_AS  = X_as_to_as_encoded
X_validation.cantidad_ataques_AS_to_AS  = X_validation.cantidad_ataques_AS_to_AS .astype('float32')

cantidad_ataques_ya_recibidos_x_ese_pais

In [ ]:
cantidad_ataques_ya_recibidos_x_ese_pais_encoder = TargetEncoder(target_type = 'binary' ,cv = 10, random_state=1)
cantidad_ataques_ya_recibidos_x_ese_pais_encoded = cantidad_ataques_ya_recibidos_x_ese_pais_encoder.fit_transform(df_train['cantidad_ataques_ya_recibidos_x_ese_pais'].to_numpy().reshape(-1, 1) , y_train)
df_train.cantidad_ataques_ya_recibidos_x_ese_pais = cantidad_ataques_ya_recibidos_x_ese_pais_encoded
df_train.cantidad_ataques_ya_recibidos_x_ese_pais = df_train.cantidad_ataques_ya_recibidos_x_ese_pais.astype('float32')

cantidad_ataques_ya_recibidos_x_ese_pais_encoded = cantidad_ataques_ya_recibidos_x_ese_pais_encoder.transform(X_validation['cantidad_ataques_ya_recibidos_x_ese_pais'].to_numpy().reshape(-1, 1))
X_validation.cantidad_ataques_ya_recibidos_x_ese_pais = X_validation.cantidad_ataques_ya_recibidos_x_ese_pais.astype('float32')

cantidad_ataques_pais_x_hora

In [ ]:
cantidad_ataques_pais_x_hora_encoder = TargetEncoder(target_type = 'binary' ,cv = 10, random_state=1)
cantidad_ataques_pais_x_hora_encoded = cantidad_ataques_pais_x_hora_encoder.fit_transform(df_train['cantidad_ataques_pais_x_hora'].to_numpy().reshape(-1, 1) , y_train)
df_train.cantidad_ataques_pais_x_hora  = cantidad_ataques_pais_x_hora_encoded
df_train.cantidad_ataques_pais_x_hora  = df_train.cantidad_ataques_pais_x_hora .astype('float32')

X_cantidad_ataques_pais_x_hora_encoded     =  cantidad_ataques_pais_x_hora_encoder.transform(X_validation['cantidad_ataques_pais_x_hora'].to_numpy().reshape(-1, 1))
X_validation.cantidad_ataques_pais_x_hora  =  X_cantidad_ataques_pais_x_hora_encoded
X_validation.cantidad_ataques_pais_x_hora  =  X_validation.cantidad_ataques_pais_x_hora .astype('float32')

freq

In [ ]:
freq_encoder = TargetEncoder(target_type = 'binary' ,cv = 10, random_state=1)
freq_encoded = freq_encoder.fit_transform(df_train['freq'].to_numpy().reshape(-1, 1) , y_train)
df_train.freq  = freq_encoded
df_train.freq  = df_train.freq .astype('float32')

X_freq_encoded =  freq_encoder.transform(X_validation['freq'].to_numpy().reshape(-1, 1))
X_validation.freq  =  X_freq_encoded
X_validation.freq  =  X_validation.freq.astype('float32')

cantidad_pais_as_attacker_encoder

In [ ]:
cantidad_pais_as_attacker_encoder = TargetEncoder(target_type = 'binary' ,cv = 10, random_state=1)
cantidad_pais_as_attacker_encoded = cantidad_pais_as_attacker_encoder.fit_transform(df_train['cantidad_pais_as_attacker'].to_numpy().reshape(-1, 1) , y_train)
df_train.cantidad_pais_as_attacker  = cantidad_pais_as_attacker_encoded
df_train.cantidad_pais_as_attacker  = df_train.cantidad_pais_as_attacker .astype('float32')

X_cantidad_pais_as_attacker_encoded =  cantidad_pais_as_attacker_encoder.transform(X_validation['cantidad_pais_as_attacker'].to_numpy().reshape(-1, 1))
X_validation.cantidad_pais_as_attacker  =  X_cantidad_pais_as_attacker_encoded
X_validation.cantidad_pais_as_attacker  =  X_validation.cantidad_pais_as_attacker.astype('float32')

#guardar

In [ ]:
import pickle
filename = 'mean_encoder_attacker_as_name.pickle'
pickle.dump(encoder_attacker_as_name ,open(filename, "wb"))

filename = 'mean_encoder_watcher_as_name.pickle'
pickle.dump(encoder_watcher_as_name, open(filename, "wb"))

filename = 'ohe_watcher_country.pickle'
pickle.dump(ohe_watcher_country, open(filename, "wb"))

filename = 'ohe_attack_type.pickle'
pickle.dump(ohe_encoder, open(filename, "wb"))

filename = 'binary_attacker_country.pickle'
pickle.dump(encoder_attacker_country ,open(filename, "wb"))

filename = 'mean_encoder_time.pickle'
pickle.dump(time_encoder, open(filename, "wb"))

filename = 'mean_encoder_shodan.pickle'
pickle.dump(shodan_encoder, open(filename, "wb"))

filename = 'mean_encoder_uuid.pickle'
pickle.dump(uuid_encoder, open(filename, "wb"))

filename = 'mean_encoder_freq.pickle'
pickle.dump(freq_encoder,open(filename, "wb"))

filename = 'mean_encoder_as_to_as.pickle'
pickle.dump(as_to_as_encoder, open(filename, "wb"))

filename = 'mean_encoder_pais_as_attacker_encoder.pickle'
pickle.dump(cantidad_pais_as_attacker_encoder, open(filename, "wb"))

filename = 'mean_encoder_ataques_pais_por_hora.pickle'
pickle.dump(cantidad_ataques_pais_x_hora_encoder, open(filename, "wb"))

filename = 'mean_encoder_cantidad_ataques_ya_recibidos_x_ese_pais.pickle'
pickle.dump(cantidad_ataques_ya_recibidos_x_ese_pais_encoder, open(filename, "wb"))

In [ ]:
import pickle
filename = 'df_train_80_20_nulos_muchoMean.pickle'
pickle.dump(df_train ,open(filename, "wb"))

filename = 'y_train_80_20_nulos_muchoMean.pickle'
pickle.dump(y_train, open(filename, "wb"))

filename = 'X_validation_80_20_nulos_muchoMean.pickle'
pickle.dump(X_validation, open(filename, "wb"))

filename = 'y_validation_80_20_nulos_muchoMean.pickle'
pickle.dump(y_validation, open(filename, "wb"))

#modelito

In [ ]:
import pickle
filename  = '/content/drive/MyDrive/data tp ml/Dataframes/df_con_features/nulos_yMUCHOmean/df_train_80_20_nulos_muchoMean.pickle'
df_train  = pickle.load(open(filename, "rb"))

filename =  '/content/drive/MyDrive/data tp ml/Dataframes/df_con_features/nulos_yMUCHOmean/y_train_80_20_nulos_muchoMean.pickle'
y_train  =   pickle.load(open(filename, "rb"))

filename     = '/content/drive/MyDrive/data tp ml/Dataframes/df_con_features/nulos_yMUCHOmean/X_validation_80_20_nulos_muchoMean.pickle'
X_validation = pickle.load(open(filename, "rb"))

filename     = '/content/drive/MyDrive/data tp ml/Dataframes/df_con_features/nulos_yMUCHOmean/y_validation_80_20_nulos_muchoMean.pickle'
y_validation = pickle.load(open(filename, "rb"))

In [ ]:
parametros = {
    'boosting_type'     :['gbdt'],
    'num_leaves'        :[8, 9, 10],
    #'max_depth'         :[3, 2, 4,],
    'learning_rate'     :[0.1, 0.2, 0.3],
    'n_estimators'      :[3,4,5],
    'subsample_for_bin' :[500, 150],
    'objective'         :['binary'],
    'class_weight'      :[None],
    'min_split_gain'    :[0.2, 0.3, 0.4],
    'min_child_weight'  :[0.3, 0.5, 0.6],
    'min_child_samples' :[3000, 5000, 7000],
    'subsample'         :[1.0, 0.5],
    'subsample_freq'    :[0, 1, 2],
    'colsample_bytree'  :[1.0, 0.5, 0.4],
    'reg_alpha'         :[0.5, 0.7],
    'reg_lambda'        :[0.5, 0.7],
    'random_state'      :[7],
    'importance_type'   :['split'],
    'metric'            :['f1']
}

parametros2 = {
    'boosting_type'     :['gbdt'],
    'num_leaves'        :[8],
    'learning_rate'     :[0.1],
    'n_estimators'      :[3],
    'subsample_for_bin' :[150],
    'objective'         :['binary'],
    'class_weight'      :[None],
    'min_split_gain'    :[0.4],
    'min_child_weight'  :[0.3],
    'min_child_samples' :[3000],
    'subsample'         :[0.5],
    'subsample_freq'    :[1],
    'colsample_bytree'  :[0.4],
    'reg_alpha'         :[0.5],
    'reg_lambda'        :[0.7],
    'random_state'      :[7],
    'importance_type'   :['split'],
    'metric'            :['f1']
}



#'max_depth'         :[3, 2, 4],

modelo = LGBMClassifier()

In [ ]:
import re
df_train = df_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [ ]:
rs = RandomizedSearchCV(modelo, param_distributions=parametros, n_iter=1, cv = 3, random_state=21, verbose=4, scoring='f1', error_score='raise')

In [ ]:
rs.fit(df_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Info] Number of positive: 362679, number of negative: 21112080
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.270631 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 381
[LightGBM] [Info] Number of data points in the train set: 21474759, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.016889 -> initscore=-4.064083
[LightGBM] [Info] Start training from score -4.064083
[CV 1/3] END boosting_type=gbdt, class_weight=None, colsample_bytree=0.4, importance_type=split, learning_rate=0.1, metric=f1, min_child_samples=7000, min_child_weight=0.6, min_split_gain=0.3, n_estimators=3, num_leaves=9, objective=binary, random_state=7, r

RandomizedSearchCV(cv=3, error_score='raise', estimator=LGBMClassifier(),
                   n_iter=20,
                   param_distributions={'boosting_type': ['gbdt'],
                                        'class_weight': [None],
                                        'colsample_bytree': [1.0, 0.5, 0.4],
                                        'importance_type': ['split'],
                                        'learning_rate': [0.1, 0.2, 0.3],
                                        'metric': ['f1'],
                                        'min_child_samples': [3000, 5000, 7000],
                                        'min_child_weight': [0.3, 0.5, 0.6],
                                        'min_split_gain': [0.2, 0.3, 0.4],
                                        'n_estimators': [3, 4, 5],
                                        'num_leaves': [8, 9, 10],
                                        'objective': ['binary'],
                                        'random_state': [7],
                                        'reg_alpha': [0.5, 0.7],
                                        'reg_lambda': [0.5, 0.7],
                                        'subsample': [1.0, 0.5],
                                        'subsample_for_bin': [500, 150],
                                        'subsample_freq': [0, 1, 2]},
                   random_state=21, scoring='f1', verbose=4)

In [ ]:
rs.best_params_

NameError: ignored

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, recall_score, accuracy_score, precision_score

In [ ]:
y_pred =rs.predict(X_validation)

In [ ]:
f1_score(y_validation, y_pred)

0.4508375865379756

In [ ]:
cm = confusion_matrix(y_validation, y_pred)
cm

array([[7832918,   67728],
       [ 106150,   71373]])

In [ ]:
import pickle

filename = 'lightgbm-0.45.pickle'
pickle.dump(rs, open(filename, "wb"))

#chequeo proporciones

In [ ]:
df_train_sin_under['label'] = y_train_sin_under
df_train_new_features['label'] = y_train_new_features

In [ ]:
y_train_sin_under = y_train_sin_under.reset_index()
y_train_sin_under.drop('index', axis=1, inplace = True)
y_train_sin_under

,label
0,0
1,0
2,0
3,0
4,0
...,...
28180963,0
28180964,0
28180965,0
28180966,0


In [ ]:
y_train_sin_under['cantidad'] = 1
chequeo1 = y_train_sin_under.groupby('label')['cantidad'].sum()
chequeo1

label
0    27721615
1      459353
Name: cantidad, dtype: int64

In [ ]:
chequeo1 = chequeo1.reset_index()


In [ ]:
total = chequeo1.cantidad.sum()
chequeo1['proporcion'] = chequeo1['cantidad'] / total
chequeo1

,label,cantidad,proporcion
0,0,27721615,0.9837
1,1,459353,0.0163


In [ ]:
y_train_new_features= y_train_new_features.reset_index()
y_train_new_features.drop('index', axis=1, inplace = True)
y_train_new_features['cantidad'] = 1
chequeo2 = y_train_new_features.groupby('label')['cantidad'].sum()
chequeo2

label
0    27545122
1      456348
Name: cantidad, dtype: int64

In [ ]:
chequeo2 = chequeo2.reset_index()

In [ ]:
total = chequeo2.cantidad.sum()
chequeo2['proporcion'] = chequeo2['cantidad'] / total
chequeo2

,label,cantidad,proporcion
0,0,27545122,0.983703
1,1,456348,0.016297
